# 필요 함수 import

In [1]:
import requests
from bs4 import BeautifulSoup

import re
from collections import Counter
import nltk
from konlpy.tag import Okt

import pandas as pd
import numpy as np

import pickle

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1차 : 쿠팡 상품 리스트 추출
> categoryId
- 밀키트 : 502482
- 1국탕전골 : 502483
- 2덮밥/비빔밥 : 502484
- 3스테이크/고기 502485
- 4면/파스타/감바스 502486
- 5분식 502487
- 6중식요리 502490
- 7기타요리 502491

In [2]:
# 쿠팡 페이지 호출 함수
def get_coupang_item(URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    res = requests.get(URL, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

## 필요 정보만 추출
- 카테고리 대분류, 상품id, 상품only, 상품명(전체), 정가, 할인율, 판매가, 100g당_가격, 별점, 리뷰수, 품절여부

In [3]:
#카테고리 대분류 라벨링 함수
def category_number(categoryId, category):
    return category.get(categoryId)


#해당 카테고리의 페이지 전체 수 가져오기
def page_number(categoryId, user_agt):
    URL = 'https://www.coupang.com/np/categories/'+categoryId+"?page=1"
    cg_soup_base = get_coupang_item(URL, user_agt)
    page_all = cg_soup_base.select('#product-list-paging > div > a')
    page_list = []
    for page_num in page_all:
        page_num = page_num.text
        try:
            page_num = int(page_num)
            page_list.append(page_num)
        except:
            pass
    return page_list[-1]


#1차 쿠팡 상품 리스트 전처리 함수
def p_list_preprocessing(cg_soup, category_num):
    #상품id, data-item-id, data-vendor-item-id
    bady_pdt = cg_soup.select_one("a.baby-product-link")
    productId = bady_pdt['data-product-id']
    itemsId = bady_pdt['data-item-id']
    vendorItemId = bady_pdt['data-vendor-item-id']

    # 상품명
    name = cg_soup.select_one("div.name").text.replace('\n', '').strip()
    name_only = name.split(', ')[0]

    # 판매가
    price_value = cg_soup.select_one("strong.price-value").text.replace(',','')
    
    try:
        # 정가
        base_price = cg_soup.select_one("del.base-price").text.replace("\n",'').strip().replace(',','')
        # 할인율
        discount_pcg = cg_soup.select_one("span.discount-percentage").text
    except:
        base_price, discount_pcg = price_value, 0

    #100g당 가격
    try:
        unit_price = cg_soup.select_one("span.unit-price").text.replace("\n", '').strip()
    except:
        unit_price = np.NaN
    
    try:
        # 별점
        star = cg_soup.select_one("span.star > em").text
        # 리뷰수
        rating_count = cg_soup.select_one("span.rating-total-count").text
        rating_count = re.sub(r'[^0-9]','',rating_count)
    except:
        star, rating_count = 0, 0

    # 품절여부 > 품절이면 1, 아니면 0
    try:
        out_of_stock = cg_soup.select_one("div.out-of-stock").text.replace("\n", '').strip()
        out_of_stock = 1
    except:
        out_of_stock = 0
    
    #DataFrame으로 병합
    coupang_item = pd.DataFrame({'카테고리명':category_num,'상품id':productId, 'data-item-id':itemsId, 'data-vendor-item-id':vendorItemId
                                , '상품':name_only,'상품명':name, '정가':base_price, '할인율':discount_pcg, '판매가':price_value
                                , '100g당_가격':unit_price, '별점':star, '리뷰수':rating_count, '품절여부':out_of_stock}
                                , index=[0])
    return coupang_item

## 쿠팡 상품 리스트 추출 실행부
> 밀키트 추가 필터링 후 전체 크롤링

- 추가 전처리 해야하는 부분
1. 100g당 가격 결측치 처리
2. 구성정보 텍스트 전처리 : 명사 추출 후 재료만 남기기
3. 구성정보 결측치 채우기 : 텍스트로 구성정보가 존재하는 경우만 >> 그외 결측치 삭제
4. 상품명에서 음식명만 추출 > 세부 카테고리 정리

In [4]:
#categoryId 기반 카테고리 대분류 정의
category = {'502483':1, '502484':2, '502485':3, '502486':4, '502487':5, '502490':6, '502491':7}
category_keys = list(category.keys())

#상품 리스트를 담을 빈 dataframe 생성
item_list = ['카테고리명','상품id','data-item-id','data-vendor-item-id','상품','상품명','정가',
             '할인율','판매가','100g당_가격','별점','리뷰수','품절여부']
coupang_items = pd.DataFrame(columns=item_list)

#쿠팡 상품 리스트 추출
cg_url = 'https://www.coupang.com/np/categories/'
user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

for categoryId in category_keys:
    page_num = page_number(categoryId, user_agt) #해당 카테고리의 페이지 마지막 페이지 번호 추출
    for page in range(1, page_num+1):
        URL = cg_url+categoryId+"?page="+str(page)+'&filter=1%23attr_12406%2419087%40DEFAULT'
        
        #페이지 호출
        cg_soup_base = get_coupang_item(URL, user_agt)
        cg_soup2 = cg_soup_base.select("#productList li")  

        #상품 리스트 추출 및 전처리
        for cg_soup in cg_soup2:
            coupang_item = p_list_preprocessing(cg_soup, category_number(categoryId, category))
            coupang_items = pd.concat([coupang_items,coupang_item], ignore_index=True)

coupang_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                373 non-null    object
 1   상품id                 373 non-null    object
 2   data-item-id         373 non-null    object
 3   data-vendor-item-id  373 non-null    object
 4   상품                   373 non-null    object
 5   상품명                  373 non-null    object
 6   정가                   373 non-null    object
 7   할인율                  373 non-null    object
 8   판매가                  373 non-null    object
 9   100g당_가격             330 non-null    object
 10  별점                   373 non-null    object
 11  리뷰수                  373 non-null    object
 12  품절여부                 373 non-null    object
dtypes: object(13)
memory usage: 38.0+ KB


In [5]:
#전체 중복행 확인
coupang_items.loc[coupang_items.duplicated(subset=['상품id']), :]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부


In [6]:
#중복행 삭제
coupang_items.drop_duplicates('상품id', inplace=True)
coupang_items.reset_index(drop=True, inplace=True)

## '100g당_가격' 결측치 채우기
> 필요시 주석 해제

In [7]:
#결측치 확인
# coupang_items.loc[coupang_items['100g당_가격'].isnull()]

# product_url = 'https://www.coupang.com/vp/products/'
# productId = 6638341720
# URL = product_url+str(productId)
# soup_100 = get_coupang_item(URL, user_agt)
# soup_100

# 2차 : 구성 정보 추출
- 상품id, 구성정보

In [8]:
#2차 : 구성정보 추출 함수
def p_c_extraction(productId, p_c_page):
    product_composition = '0'
    for item in p_c_page:
        s = item.text.replace('\n','').strip()
        if product_composition == '2': #3: 재료부분만 추출
            product_composition = s
        elif product_composition =='1':#2: 불필요한 text 제외
            product_composition='2'
        elif '구성 정보' in s :         #1: '구성 정보' 존재여부 확인
            product_composition = '1'

    #4: 없으면 결측값 처리
    if product_composition.isdigit():
        product_composition = np.NaN
    
    data_item = pd.DataFrame({'상품id':productId, '구성정보':product_composition}, index=[0])
    return data_item

## 2차 실행부

In [9]:
#2차 실행부
data_items = pd.DataFrame(columns=['상품id', '구성정보'])

for idx in range(len(coupang_items)):
    #각 상품 상세페이지 호출
    product_url = 'https://www.coupang.com/vp/products/'
    productId = coupang_items.loc[idx, '상품id'] #data-product-id
    itemsId = coupang_items.loc[idx, 'data-item-id']  #data-item-id
    vendorItemId = coupang_items.loc[idx, 'data-vendor-item-id'] #data-vendor-item-id
    URL = product_url+str(productId)+'/items/'+str(itemsId)+'/vendoritems/'+str(vendorItemId)
    
    pdt_soup = get_coupang_item(URL, user_agt)
    
    #'구성정보' 추출
    p_c_page = pdt_soup.find('body').children
    data_item = p_c_extraction(productId, p_c_page)
    data_items = pd.concat([data_items, data_item], ignore_index=True)

data_items.head()

,상품id,구성정보
0,1866720935,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
1,293421715,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채..."
2,1717552921,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성..."
3,4926044090,NaN
4,2202010823,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다."


## 결측치에 곰곰 채우기

In [10]:
#곰곰 상품id 추출
gomgom_productId = list(coupang_items.loc[coupang_items['상품'].str.contains('곰곰')]['상품id'].values)

#곰곰 이미지에서 추출한 구성정보 list
with open('../../data/gom_typo_fix.pkl', 'rb') as fr:
    gomgom_data = pickle.load(fr)

#상품id + 구성정보
gomgom_df = pd.DataFrame(zip(gomgom_productId, gomgom_data), columns=['상품id', '구성정보'])

### 곰곰 결측치 채우기
for i in range(len(gomgom_df)):
    idx = data_items[data_items['상품id'] == gomgom_productId[i]].index.values
    gomgom = gomgom_df.loc[i,'구성정보']
    data_items.loc[idx, '구성정보'] = ', '.join(gomgom)
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    373 non-null    object
 1   구성정보    330 non-null    object
dtypes: object(2)
memory usage: 6.0+ KB


## 구성정보 전처리 후 재대입

In [11]:
#구성정보 재료 전처리
def p_c_preprocessing(product_composition):
    product_composition = re.sub(r'\\n', '', product_composition)
    product_composition = re.sub(r'\)', ',', product_composition)
    product_composition = re.sub(r'[^ㄱ-ㅣ가-힣a-zA-Z\,\(\:]', ' ', product_composition)
    product_composition = product_composition.strip()
    
    replace_list = ['으로 구성되어 있습니다','으로 구성되어있습니다','로 구성되어 있습니다','로 구성되어 있어요','로 구성되어있습니다']
    for r in replace_list:
        product_composition = product_composition.replace(r, '')
    
    replace_list = ['과', '와']
    for r in replace_list:
        product_composition = product_composition.replace(r, ',') 
    
    # 추가 작업에 따라 추가 여부 결정
    # if ('야채' in product_composition) or ('채소(' in product_composition) or ('채소 :' in product_composition):
    #     replace_list = ['야채 세트', '혼합 채소', '혼합 해물', '채소(', '채소 :']
    #     for r in replace_list:
    #         product_composition = re.sub('[\(\)\:]', ',', product_composition)
    #         product_composition = product_composition.replace(r, '')
    # else:
    product_composition = re.sub('[\(\)\:]', ',', product_composition)

    return product_composition

In [12]:
#요리재료내용 결측치 삭제
data_items_drop = data_items[data_items.loc[:,'구성정보'].isnull()].index
data_items_drop = data_items.drop(data_items_drop)
data_items_drop.reset_index(drop=True, inplace=True)
data_items_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품id    330 non-null    object
 1   구성정보    330 non-null    object
dtypes: object(2)
memory usage: 5.3+ KB


>> 소고기 == 쇠고기 이건 어디에..?

In [13]:
#요리재료_전처리(new column) 
data_items_drop['구성정보_전처리'] = np.NaN

for idx in range(len(data_items_drop)):
    ingredient = data_items_drop.loc[idx, '구성정보']
    ingredients = p_c_preprocessing(ingredient)
    ingredients = ingredients.split(',')
    
    ingre_lists = []
    for i in ingredients:
        i = i.strip()
        replace_list = ['칼국수 면', '다진 마늘', '대구 곤이', '명태 곤이']
        if i in replace_list:
            i = i.split(' ')
            i = ''.join(i)

        i = i.split(' ')
        if i[-1] == '등' :  # 등을 그대로 두는 것이 나을지 지금처럼 빼는게 나을지..?
            i.pop(-1)
            i = ' '.join(i)
            ingre_lists.append(i)
        else:
            i = ' '.join(i)
            ingre_lists.append(i)
        
    #null값 제거
    ingre_lists = list(filter(None, ingre_lists))

    #요리재료_전처리(new column)에 추가
    data_items_drop.loc[idx, '구성정보_전처리'] = ', '.join(ingre_lists)

data_items_drop.iloc[10:20, :]

,상품id,구성정보,구성정보_전처리
10,2251401014,"두부, 청국장, 애호박, 돼지고기, 대파, 양파, 혼합장","두부, 청국장, 애호박, 돼지고기, 대파, 양파, 혼합장"
11,2322032858,"돼지고기, 모듬햄, 대파, 양파, 양배추, 라면사리, 부대찌개소스, 김치, 슬라이스...","돼지고기, 모듬햄, 대파, 양파, 양배추, 라면사리, 부대찌개소스, 김치, 슬라이스..."
12,6617266915,"소 스지와 전각 슬라이스, 토자연 우사골육수 2팩, 참소스 2팩으로 구성되어 있습니다.","소 스지, 전각 슬라이스, 토자연 우사골육수 팩, 참소스 팩"
13,188130043,"소고기, 소스, 쥬키니호박, 팽이버섯, 대파, 청양고추, 두부로 구성되어 있습니다.","소고기, 소스, 쥬키니호박, 팽이버섯, 대파, 청양고추, 두부"
14,6579540231,"왕만두, 소스, 칼국수 면, 혼합 채소(느타리버섯, 대파)로 구성되어 있습니다.","왕만두, 소스, 칼국수면, 혼합 채소, 느타리버섯, 대파"
15,6641459417,"홍합, 문어, 오징어, 반각 가리비, 꽃게, 명태알, 대구 곤이, 새우, 야채 세트...","홍합, 문어, 오징어, 반각 가리비, 꽃게, 명태알, 대구곤이, 새우, 야채 세트,..."
16,5026557182,"미역, 쇠고기, 육수, 볶음, 맛기름","미역, 쇠고기, 육수, 볶음, 맛기름"
17,235501175,"소시지, 소스, 양파, 대파 등으로 구성되어 있습니다. \n\n","소시지, 소스, 양파, 대파"
18,5026557256,"두부, 쇠고기, 소스, 팽이버섯, 청양고추, 애호박, 대파, 감자","두부, 쇠고기, 소스, 팽이버섯, 청양고추, 애호박, 대파, 감자"
19,320547561,"누룽지, 해산물 팩, 단호박, 청경채, 콩기름, 소스, 야채팩으로 구성되어 있습니다.","누룽지, 해산물 팩, 단호박, 청경채, 콩기름, 소스, 야채팩"


# 비교를 위한 리스트 생성

In [14]:
# 만개의 레시피 재료 목록 read
ingre_10000 = []
with open('./만개의레시피_재료_목록.txt', 'r') as f :
    lines = f.readlines()
    lines = str(lines).split(',')
    for line in lines:
        ingre_10000.append(line) 
len(ingre_10000)

37970

In [15]:
#쿠팡 재료 목록 생성
ingre_coupang_list = data_items_drop['구성정보_전처리'].dropna()
ingre_coupang_lists = ingre_coupang_list.to_list()

ingre_coupangs = []
for item in ingre_coupang_lists:
    ingre_coupang=[]
    items = item.split(', ')
    for i in items:
        ingre_coupang.append(i)
    ingre_coupangs.append(ingre_coupang)

# #쿠팡 재료 목록 저장
# with open('쿠팡_재료_목록.txt', 'w') as f :
#     for ingre in ingre_coupang:
#         f.write(ingre)

In [16]:
#쿠팡에만 있는 재료만
ingre_not_10000 = []
for items in ingre_coupangs :
    not_10000 = []
    for i in items:
        if not i in ingre_10000 and i != '':
            not_10000.append(i)
    ingre_not_10000.append(not_10000)
print(len(ingre_not_10000))
ingre_not_10000

330


[[],
 ['소스  종', '표고버섯은 중량 기준으로 담겨있어 수량이 상이할 수 있어요'],
 [],
 [],
 [],
 ['기타 수산물가공품'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['토자연 우사골육수  팩', '참소스  팩'],
 [],
 ['혼합 채소'],
 ['반각 가리비', '야채 세트', '야채 세트'],
 [],
 [],
 [],
 ['해산물 팩', '야채팩'],
 [],
 ['낙곱새전골 소스', '낙지곱창'],
 [],
 ['바지락맛 액상 소스'],
 ['잡채 면', '간장 배 양념'],
 [],
 [],
 ['야채 세트', '야채 세트'],
 ['야채팩', '청국장  조리 시 물    mL는 추가로 준비해주세요'],
 ['부대찌개 소스'],
 ['정통 순두부찌개순두부', '순두부 찌개양념장'],
 [],
 ['디포리 국물용팩'],
 ['야채 세트', '액상소스'],
 ['야채 세트', '바른 마늘'],
 ['반 가리비', '해물탕 전용 소스'],
 ['혼합 채소', '혼합 해물', '자숙홍합살'],
 [],
 [],
 ['야채세트'],
 [],
 [],
 ['맑은 지리탕 소스'],
 [],
 ['소고기 포장육', '소스  종'],
 ['백목이버섯'],
 ['바지락 맛 액상 소스'],
 ['야채 세트', '야채 세트'],
 [],
 [],
 [],
 ['야채세트', '바른마늘'],
 ['야채 세트', '바른 마늘'],
 ['주름 미더덕', '얼큰 분말 스프'],
 ['수산물가공품'],
 ['야채 세트'],
 ['기타 수산물가공품'],
 ['분말 수프'],
 ['알탕 전용 소스'],
 ['다채로운 모양의 어묵들'],
 ['야채 세트'],
 [],
 ['야채세트', '바른마늘'],
 [],
 [],
 [],
 [],
 ['기타 수산물가공품'],
 ['야채 세트', '바른 마늘'],
 [],
 ['야채세트'],
 ['야채 세트', '야채 세트'],
 ['야채 세트'],
 ['닭한마리 소스', '딥소스'],
 [],
 ['야채 세

In [17]:
#일치하는 재료 리스트로
ingre_10000s = []
for items in ingre_coupangs :
    ingre_10000_list = []
    for i in items:
        if i in ingre_10000:
            ingre_10000_list.append(i)
    ingre_10000s.append(ingre_10000_list)
print(len(ingre_10000s))
ingre_10000s

330


[['양념육',
  '소곱창',
  '소대창',
  '전골용 육수',
  '새송이버섯',
  '느타리버섯',
  '소고기',
  '두부',
  '배추',
  '양파',
  '깻잎',
  '라면사리'],
 ['소고기', '만두', '생면', '육수', '표고버섯', '팽이버섯', '채소', '배추', '청경채', '깻잎'],
 ['소시지', '오뗄팜S', '프레스햄', '소고기', '치즈', '라면사리', '채소', '김치', '소스'],
 ['쇠고기',
  '새송이버섯',
  '표고버섯',
  '초간장소스',
  '칠리소스',
  '샤브용육수',
  '생면',
  '칼국수면',
  '숙면',
  '감자수제비',
  '깻잎',
  '팽이버섯',
  '배추',
  '청경채'],
 ['해물', '채소', '연두부', '다시팩', '소스'],
 ['명태알', '대구곤이', '새우', '채소', '무', '대파', '청양고추', '소스'],
 ['순두부', '애호박', '소고기', '팽이버섯', '대파', '소스'],
 ['애호박', '양파', '대파', '청양고추', '두부', '팽이버섯'],
 ['소고기', '당면', '팽이버섯', '소스', '참기름'],
 ['돼지고기', '야채', '애호박', '감자', '느타리버섯', '양파', '청양고추', '대파', '소스'],
 ['두부', '청국장', '애호박', '돼지고기', '대파', '양파', '혼합장'],
 ['돼지고기', '모듬햄', '대파', '양파', '양배추', '라면사리', '부대찌개소스', '김치', '슬라이스', '치즈'],
 ['소 스지', '전각 슬라이스'],
 ['소고기', '소스', '쥬키니호박', '팽이버섯', '대파', '청양고추', '두부'],
 ['왕만두', '소스', '칼국수면', '느타리버섯', '대파'],
 ['홍합', '문어', '오징어', '꽃게', '명태알', '대구곤이', '새우', '해물탕 양념', '무', '파', '고추'],
 ['미역', '쇠고기', '육수', '볶음

## 긴 텍스트에서 사전과 일치하는 요리재료만 추출

In [18]:
#요리재료 사전과 일치여부를 통해 요리재료 추출
ingredients_list = []
for coupangs in ingre_coupangs :
    ingre_list = []
    for coupang in coupangs:
        if coupang in ingre_10000:
            ingre_list.append(coupang)
        else:
            ingre = [ man for man in ingre_10000 if man in coupang]
            ingre_list.extend(ingre)
    ingredients_list.append(ingre_list)
    print(ingre_list)
#ingredients_list

['양념육', '소곱창', '소대창', '전골용 육수', '새송이버섯', '느타리버섯', '소고기', '두부', '배추', '양파', '깻잎', '라면사리']
['소고기', '만두', '생면', '소', '소스', '육수', '표고버섯', '팽이버섯', '채소', '배추', '청경채', '깻잎', '표고', '기', '버섯', '표고버섯', '중']
['소시지', '오뗄팜S', '프레스햄', '소고기', '치즈', '라면사리', '채소', '김치', '소스']
['쇠고기', '새송이버섯', '표고버섯', '초간장소스', '칠리소스', '샤브용육수', '생면', '칼국수면', '숙면', '감자수제비', '깻잎', '팽이버섯', '배추', '청경채']
['해물', '채소', '연두부', '다시팩', '소스']
['기', '기타', '물', '산', '명태알', '대구곤이', '새우', '채소', '무', '대파', '청양고추', '소스']
['순두부', '애호박', '소고기', '팽이버섯', '대파', '소스']
['애호박', '양파', '대파', '청양고추', '두부', '팽이버섯']
['소고기', '당면', '팽이버섯', '소스', '참기름']
['돼지고기', '야채', '애호박', '감자', '느타리버섯', '양파', '청양고추', '대파', '소스']
['두부', '청국장', '애호박', '돼지고기', '대파', '양파', '혼합장']
['돼지고기', '모듬햄', '대파', '양파', '양배추', '라면사리', '부대찌개소스', '김치', '슬라이스', '치즈']
['소 스지', '전각 슬라이스', '육수', '우', '사골육수', '골', '우사골', '팩', '연', '사골', '참', '참소스', '소', '팩', '소스']
['소고기', '소스', '쥬키니호박', '팽이버섯', '대파', '청양고추', '두부']
['왕만두', '소스', '칼국수면', '채', '혼합', '채소', '혼', '소', '느타리버섯', '대파']
['홍합', '문어', 

['소고기', '숙주나물', '표고버섯', '폰즈 소스', '농축 육수', '칠리소스', '깻잎', '배추', '청경채', '팽이버섯', '육수', '칼국수 면', '표고', '기', '국', '버섯', '면', '표고버섯', '국수', '중', '국수 면', '칼', '칼국수']
['조피볼락', '무', '양파', '대파', '양념', '양', '탕', '매운', '매']
['라면사리', '육수', '치즈', '숙주나물', '프레스 햄', '돼지고기', '다진마늘', '대파']
['오리고기', '들깨', '탕', '깨', '소', '들', '소스']
['라면사리', '대파채', '숙주나물', '프레스 햄', '소시지', '소스', '치즈']
['해물', '오징어', '새우', '피홍합', '소스', '누룽지']
['팽이버섯', '콩나물', '꽃게', '양파', '애호박', '대파', '새우', '청양고추', '느타리버섯', '무', '양념']
['우삼겹', '순대', '곱창', '숙면', '팽이버섯', '양념장', '청경채잎', '깻잎', '양배추', '대파', '양파']
['새꼬막', '소스', '양배추', '적양배추', '당근', '고추', '참기름']
['멍게', '소스', '양배추', '적양배추', '당근', '고추', '참기름']
['쇠고기', '양파', '홍피망', '청피망', '쥬키니호박', '총알새송이버섯', '방울토마토', '마늘종', '단호박']
['돼지고기', '빵', '노랑 파프리카', '빨강 파프리카', '청피망', '양파', '브로콜리', '수프', '포테이토', '포', '컵', '스테이크 씨즈닝', '조', '신', '향신료', '올리브오일', '소스']
['소고기', '소고기 갈비', '기', '갈비찜', '찜', '중', '갈비', '소', '갈', '고기', '소고기', '육전', '기', '소', '전', '고기', '소고기', '기', '채', '고기 버섯', '버섯', '소', '잡채', '고기']
['기', '불고기'

['홍합', '가리비', '홍', '오징어', '꽃게', '중화면', '채', '야채', '세', '고추', '파', '양파', '기름', '기', '고추', '맛', '짬뽕 소스']
['오징어', '소스']
['꽃게', '새우', '바지락', '홍합', '소스', '생면', '채', '야채', '세', '채', '야채', '세', '양파', '파', '고추']
['교자', '우동면', '냉동야채', '배추', '대파', '새송이버섯', '표고버섯', '목이버섯', '육수', '육개장', '개', '장']
['닭갈비', '숙면', '모짜렐라 치즈']
['종합 어묵', '삼', '양', '소', '양지', '우동면', '냉동야채', '배추', '대파', '새송이버섯', '표고버섯', '목이버섯', '소스']
['흰다리새우살', '빵', '바질', '감', '소', '소스']
['오징어', '소스']
['오징어', '초', '홍합', '초록', '홍', '새우살', '새우', '우', '점보 칵테일 새우', '칵테일', '칵테일 새우살', '새', '살', '칵테일 새우', '소스']
['홍가리비', '홍합', '채', '야채', '세', '양파', '마늘', '건고추', '올리브유', '버터', '허브 솔트']
['갈치', '용', '조림', '양념', '양', '조', '생', '조림용', '생선']
['쌀', '국', '쌀국수', '면', '국수', '국수 면', '고명', '육수', '소', '소스']
['피조개살', '비빔소스', '매', '매콤', '소', '소스', '참기름']
['포', '용', '백', '백숙용', '누룽지', '장', '닭고기', '누룽지', '기', '용', '백', '백숙용', '황', '황기', '치킨스톡']
['쌀', '국', '쌀국수', '면', '국수', '국수 면', '고명', '육수', '소', '소스']
['손', '손만두', '기', '만두', '고기', '소고기', '채소', '청경채', '대파', '청양고추'

# 쿠팡 상품 리스트와 구성정보 병합

In [19]:
#쿠팡 상품 리스트와 구성정보 병합
coupang = pd.merge(coupang_items, data_items_drop)
coupang.to_csv('coupang_filtering.csv')
coupang.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   카테고리명                330 non-null    object
 1   상품id                 330 non-null    object
 2   data-item-id         330 non-null    object
 3   data-vendor-item-id  330 non-null    object
 4   상품                   330 non-null    object
 5   상품명                  330 non-null    object
 6   정가                   330 non-null    object
 7   할인율                  330 non-null    object
 8   판매가                  330 non-null    object
 9   100g당_가격             289 non-null    object
 10  별점                   330 non-null    object
 11  리뷰수                  330 non-null    object
 12  품절여부                 330 non-null    object
 13  구성정보                 330 non-null    object
 14  구성정보_전처리             330 non-null    object
dtypes: object(15)
memory usage: 41.2+ KB


In [20]:
coupang.head()

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,7%,24210,"(100g당 1,519원)",4.5,11258,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ..."
1,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49199,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기, 만두, 생면, 소스 종, 육수, 표고버섯, 팽이버섯, 채소, 배추, 청경..."
2,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",20810,28%,14900,"(100g당 1,717원)",5.0,36321,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스"
3,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15635,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면..."
4,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12047,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물, 채소, 연두부, 다시팩, 소스"


In [21]:
#구성정보 결측치 확인
print('결측치 개수: ',coupang.구성정보.isnull().sum())
coupang.loc[coupang['구성정보'].isnull()]

결측치 개수:  0


,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리


# 상세카테고리 추가

In [22]:
def category_name(product):
    if '전골' in product : name = '전골'
    elif '나베' in product : name= '나베'
    elif '탕' in product : name= '탕'
    elif '국' in product : name= '국'
    elif '국수' in product : name= '국수'
    elif '스테이크' in product : name= '스테이크'
    elif '파스타' in product : name= '파스타'
    elif '감바스' in product : name= '감바스'
    elif '소바' in product : name= '소바'
    elif '면' in product : name= '면'
    elif '비빔밥' in product : name= '비빔밥'
    else: name='기타'
        
    return name

In [23]:
coupang['상세카테고리'] = coupang['상품'].apply(lambda x : category_name(x))
coupang.head()

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,7%,24210,"(100g당 1,519원)",4.5,11258,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육, 소곱창, 소대창, 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...",전골
1,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49199,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기, 만두, 생면, 소스 종, 육수, 표고버섯, 팽이버섯, 채소, 배추, 청경...",나베
2,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",20810,28%,14900,"(100g당 1,717원)",5.0,36321,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스",전골
3,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15635,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...",나베
4,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12047,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물, 채소, 연두부, 다시팩, 소스",탕


In [24]:
coupang['상세카테고리'].value_counts()

기타      127
탕        70
국        37
전골       25
파스타      25
스테이크     18
감바스      14
면         7
나베        4
비빔밥       3
Name: 상세카테고리, dtype: int64

# 상세페이지 이미지에서 구성정보 추출
- 텍스트 추출 가능한 브랜드 : 곰곰, 딜리조이, 프렙

In [25]:
# 상품 상세페이지 - 상세정보 이미지만 링크 추출 코드
def get_coupang_image(img_URL, user_agt):
    headers = {"User-Agent":user_agt
               , "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
    page = requests.get(img_URL, headers= headers).text
    #coupang_image_soup = BeautifulSoup(page.text, "html.parser")
    #return coupang_image_soup
    return page

In [26]:
coupang.loc[coupang['상품'].str.contains('딜리조이')]

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리


In [27]:
image_productId = list(coupang.loc[coupang['상품'].str.contains('곰곰|딜리조이|프렙')]['상품id'].values)
image_dataitemId = list(coupang.loc[coupang['상품'].str.contains('곰곰|딜리조이|프렙')]['data-item-id'].values)
image_datavendorId = list(coupang.loc[coupang['상품'].str.contains('곰곰|딜리조이|프렙')]['data-vendor-item-id'].values)

image_links = []
for product in range(len(image_productId)):
    img_URL = product_url + str(image_productId[product]) + '/items/' +str(image_dataitemId[product])+'/vendoritems/'+str(image_datavendorId[product])
    user_agt = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46"

    page = get_coupang_image(img_URL, user_agt)

    link_start = []
    for text in re.finditer('//thumbnail',page):
        link_start.append(text.start())
        
    link_end = []
    for text in re.finditer('g"}],',page):
        link_end.append(text.start())
    
    for num in range(len(link_start)):
        image_links.append('https:' + page[int(link_start[num]):int(link_end[num]) + 1])

In [28]:
print(len(image_links))

112


> 온라인 이미지를 바로 볼 수 있는 코드 참고
https://lapina.tistory.com/73

In [29]:
import cv2
import numpy as np
import urllib.request
import pytesseract

def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype='uint8')
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  return image

In [30]:
brand_items = []
for im_link in image_links:
  image = url_to_image(im_link)
  rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 영어 + 한글 혼합된 텍스트 추출
  text = pytesseract.image_to_string(rgb_image, lang='kor')
  brand_items.append(text)

In [31]:
# 이미지 다운로드 코드
# import urllib.request
# count = 0
# for i in image_link:
#     count += 1
#     savename = "imtest" + str(count) + ".jpg"

#     # url이 가리키는 주소에 접근해서 해당 자원을 로컬 컴퓨터에 저장하기
#     urllib.request.urlretrieve(i, savename)

# print("저장완료!!!")
# 우선 전체 결측값 링크 넣어봤더니 217개의 이미지 나옴
# 각 링크별로 이미지가 평균 6-7개정도 있는데 구별 용이하게 따로 리스트로 묶어주던지, 
# 곰곰, 딜리조이, 프렙만 추출하는 코드로 전처리 다시 하면 될듯

- https://yunwoong.tistory.com/73
- https://blog.naver.com/PostView.nhn?blogId=beodeulpiri&logNo=221615329276

In [32]:
# 이미지에서 텍스트 추출 코드 with pytesserect

# pytesseract github에서 설치하고, 설치 폴더 경로 환경변수에 추가해줘야 합니다. 
# 설치할때 language 에서 Korean 체크 해야함

# pip install pytesseract
# pip install opencv-python (cv2 모듈)

In [33]:
import pytesseract
import cv2 

file = './imtest1.jpg'
image = cv2.imread(file)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 영어 + 한글 혼합된 텍스트 추출
text = pytesseract.image_to_string(rgb_image, lang='kor')
print(text)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
